In [1801]:
import pyodbc
import pandas as pd

symbol = 'intc'
conn = pyodbc.connect(
    "DRIVER={SQL Server};SERVER=lenovo-desktop;DATABASE=Qihuo;UID=samtsql;PWD=F(W}q:TsyK,7^+>`P28e79s#Uc5n")
query = f"""select m1.date as 'Date',DATEPART(hour, m1.Date) AS 'Hour', m1.[Close] as 'Market', m2.[close] as 'Stock'  from MinuteQuote m1
inner join MinuteQuote m2 on m1.Date = m2.date
where m1.Date > '2002-04-08 01:20:00' and m1.Contract = 'ym' and m2.Contract = '{symbol}'
and DATEPART(minute, m1.Date) % 10 = 0
order by m1.date"""

df = pd.read_sql(query, conn)
df.set_index('Date', inplace=True)


c:\Users\SHSIEH\Documents\ProjectsFuture\ML\project-walkthroughs\stock\stock_env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [1802]:
df["StockReturn"] = df.rolling(2).apply(lambda x: x.iloc[1] / x.iloc[0] - 1)["Stock"]
df["MarketReturn"] = df.rolling(2).apply(lambda x: x.iloc[1] / x.iloc[0] - 1)["Market"]
df["OutPerform"] = df["StockReturn"] - df["MarketReturn"]
df["Target"] = (df.apply(lambda x: x > 0)["OutPerform"]).astype(int)

# df.plot.line(y="OutPerform", use_index=True)

In [1803]:
predictors = ['Hour']
for i in range(12):
    df[f'OutPerform{pow(2,i)*5}'] = df['OutPerform'].rolling(pow(2,i)).sum()
    predictors.append(f'OutPerform{pow(2,i)*5}')
    df[f'StockReturn{pow(2,i)*5}'] = df['StockReturn'].rolling(pow(2,i)).sum()
    predictors.append(f'StockReturn{pow(2,i)*5}')


In [1804]:
prev = df.copy()
prev = prev.shift(1)
data = df[["OutPerform","Target"]]


In [1805]:
data = data.join(prev[predictors])
data = data.copy().dropna()


In [1806]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import precision_score

model = RandomForestClassifier(n_estimators=100, min_samples_split=1200, random_state=1)


In [1807]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds > .6] = 1
    preds[preds<=.6] = 0
    print(preds)
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [1808]:
def backtest(data, model, predictors, start=50000, step=10000):
    all_predictions = []
    loop = 0
    for i in range(start, data.shape[0], step):
        train = data.iloc[loop * step:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
        loop = loop + 1

    return pd.concat(all_predictions)


In [1809]:
data.shape[0]

199767

In [1810]:
predictions = backtest(data, model, predictors)

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]


In [1811]:
print(predictions)

                     Target  Predictions
Date                                    
2007-06-20 10:10:00       0          0.0
2007-06-20 10:20:00       0          0.0
2007-06-20 10:30:00       0          0.0
2007-06-20 10:40:00       0          0.0
2007-06-20 10:50:00       1          0.0
...                     ...          ...
2022-05-26 12:10:00       1          0.0
2022-05-26 12:20:00       0          0.0
2022-05-26 12:30:00       0          0.0
2022-05-26 12:40:00       1          0.0
2022-05-26 12:50:00       1          0.0

[149767 rows x 2 columns]


In [1812]:
predictions["Predictions"].value_counts()

0.0    149767
Name: Predictions, dtype: int64

In [1813]:
precision_score(predictions["Target"], predictions["Predictions"])

c:\Users\SHSIEH\Documents\ProjectsFuture\ML\project-walkthroughs\stock\stock_env\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0

In [1814]:
import pickle
pickle.dump(model, open(f"{symbol}", 'wb'))

In [1815]:
predictions.to_csv(f"{symbol}.csv")

In [1816]:
data

,OutPerform,Target,Hour,OutPerform5,StockReturn5,OutPerform10,StockReturn10,OutPerform20,StockReturn20,OutPerform40,...,OutPerform640,StockReturn640,OutPerform1280,StockReturn1280,OutPerform2560,StockReturn2560,OutPerform5120,StockReturn5120,OutPerform10240,StockReturn10240
Date,,,,,,,,,,,,,,,,,,,,,
2002-06-19 08:20:00,0.005349,1,8.0,0.003586,0.007911,0.002981,0.008703,0.006007,0.009503,0.003916,...,-0.026190,0.014152,-0.033997,-0.036914,-0.235327,-0.278392,-0.213992,-0.261537,-0.236137,-0.290124
2002-06-19 08:30:00,0.001935,1,8.0,0.005349,0.007849,0.008935,0.015761,0.009440,0.014183,0.009450,...,-0.018297,0.045792,-0.024634,-0.023805,-0.227808,-0.269965,-0.216812,-0.263045,-0.243088,-0.297979
2002-06-19 08:40:00,-0.008046,0,8.0,0.001935,0.006231,0.007283,0.014080,0.010264,0.022783,0.013714,...,-0.014667,0.038213,-0.017778,-0.010022,-0.228102,-0.266626,-0.216036,-0.257973,-0.232149,-0.284570
2002-06-19 08:50:00,0.002801,1,8.0,-0.008046,-0.009288,-0.006112,-0.003057,0.002823,0.012703,0.004315,...,-0.030148,0.019310,-0.017325,-0.007894,-0.235579,-0.277068,-0.224508,-0.266104,-0.232574,-0.286627
2002-06-19 09:00:00,-0.004117,0,8.0,0.002801,0.003906,-0.005245,-0.005382,0.002038,0.008698,0.008045,...,-0.022525,0.030359,-0.007793,0.005250,-0.231003,-0.270857,-0.222256,-0.263935,-0.229604,-0.282161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-26 12:10:00,0.000613,1,12.0,-0.001923,-0.002750,-0.002336,-0.002980,0.002196,0.000238,-0.000620,...,0.012160,0.037421,-0.020211,0.004203,-0.001583,0.008330,-0.025262,-0.097197,-0.006073,-0.011306
2022-05-26 12:20:00,-0.000412,0,12.0,0.000613,0.000919,-0.001311,-0.001831,0.000972,0.000697,0.000574,...,0.012732,0.040249,-0.017691,0.006275,-0.002253,0.009018,-0.023379,-0.095035,-0.003080,-0.009039
2022-05-26 12:30:00,-0.000537,0,12.0,-0.000412,-0.001607,0.000201,-0.000688,-0.002136,-0.003668,0.002474,...,0.012208,0.037208,-0.017109,0.005130,-0.002207,0.008797,-0.025205,-0.099337,-0.003464,-0.011771


In [1817]:
tempData = data[['OutPerform']].copy()
# tempPredictions = predictions.copy()


In [1818]:
result1 = pd.merge(tempData, predictions, left_index=True, right_index=True)

In [1819]:
from datetime import datetime
result1['year'] = result1.index.strftime('%Y')

In [1820]:
result1.loc[result1['Predictions'] == 1]

,OutPerform,Target,Predictions,year
Date,,,,


In [1821]:
resultsum = result1.groupby('year').sum()
resultmean = result1.groupby('year').mean()
print(resultsum)


      OutPerform  Target  Predictions
year                                 
2007    0.177948    2655          0.0
2008    0.014015    5036          0.0
2009    0.084687    4900          0.0
2010   -0.099505    4926          0.0
2011    0.092382    5032          0.0
2012   -0.226368    4922          0.0
2013   -0.008616    4938          0.0
2014    0.289332    5026          0.0
2015    0.005311    4933          0.0
2016   -0.055808    4978          0.0
2017    0.040500    4923          0.0
2018    0.143729    5039          0.0
2019    0.086677    5078          0.0
2020   -0.180838    4968          0.0
2021   -0.082974    4976          0.0
2022   -0.039471    1989          0.0


In [1822]:
print(resultmean)

        OutPerform    Target  Predictions
year                                     
2007  3.357501e-05  0.500943          0.0
2008  1.393131e-06  0.500596          0.0
2009  8.433237e-06  0.487951          0.0
2010 -9.890131e-06  0.489613          0.0
2011  9.183102e-06  0.500199          0.0
2012 -2.276433e-05  0.494972          0.0
2013 -8.609054e-07  0.493405          0.0
2014  2.887262e-05  0.501547          0.0
2015  5.287400e-07  0.491139          0.0
2016 -5.546415e-06  0.494733          0.0
2017  4.048364e-06  0.492103          0.0
2018  1.439307e-05  0.504606          0.0
2019  8.645270e-06  0.506483          0.0
2020 -1.794028e-05  0.492857          0.0
2021 -8.246323e-06  0.494534          0.0
2022 -9.850564e-06  0.496381          0.0
